In [10]:
! pip install chromadb -q # vector db
! pip install pillow
! pip install open-clip-torch -q # embedding model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 34.

In [11]:
from transformers import pipeline
pipe=pipeline("image-to-text",model="jinhybr/OCR-Donut-CORD")
import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction # for multimodal capabilities
from chromadb.utils.data_loaders import ImageLoader

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [23]:
# create a collection (this is a vector database)
image_loader=ImageLoader() # loads the images
multimodal_ef= OpenCLIPEmbeddingFunction() # converts the images to embeddings
multimodal_db = chroma_client.get_or_create_collection(name="multimodal_db",
                                                       embedding_function=multimodal_ef,
                                                       data_loader=image_loader)

In [14]:
import re
def clean_text(dirty_text):
  text=dirty_text[0]['generated_text']
  # use regular expressions to clean the output of the OCR
  cleaned_text= re.sub(r"<[^>]*>", "", text)
  return cleaned_text



In [15]:
from PIL import Image
import os
metadatas=[]
uris=[]
ids=[]
def read_and_convert(folder_path):
  file_data =[]
  for file_name in os.listdir(folder_path):
    file_path=os.path.join(folder_path,file_name)
    # get the image from the folder and read it using Image
    text=pipe(Image.open(file_path))
    # save the read text as metadata
    text=clean_text(text)

    metadatas.append({"content":text})
    # add the file path as a uri for extraction later
    uris.append(file_path)
  return metadatas

folder_path="images"
file_data=read_and_convert(folder_path)




In [16]:
for data in file_data:
  print(f"Text : {data['content']} \n")

Text :  KINGLEY Street 34 WTB 50H ZA177468-POS-03 15:20 ZAL174 26 Feb 2016) 34 1532 Meantine Pole = 10.20 10.20 2 10.20 Food and Drink 10.20 10.20 VAT Net 2.50 8.50 20% VAT 2.70 20% VAT Total 10.20 10.20 10.20 10.20 1.70 Receipt 5.0 1 0.20 0207 494 0834 895 232 

Text :  Restaurant Name 3 Address 1 1 Address 2 3/15/12 6:06:44 PM 3 John 3 Table: 201 Sec. S Guest 1 1 #11 BLOODY MARY 5 40.75 FRESH OYSTER 1 12.50 553.25 5.33 664.60 664.60 0sI 

Text :  CITY of FALMOUTH 06/18/2003 PAYMENT BECEIPT 0123 0000 TAX 06/18/2003 MURBHY, CHRISTIAN E. Checck Number (s) : 001234 ACTC # M4751R / TRIH* 0019 02 02 629.95 02 Tx 0103 02 02 629.95 0103 05 05 29.53 1289.43 1289.43.00 1259.90.00 1259.90.00 



In [17]:
for uri in uris:
  print(uri + "\n")

images/receipt.jpg

images/receipt-3.jpg

images/receipt-2.png



In [18]:
metadatas

[{'content': ' KINGLEY Street 34 WTB 50H ZA177468-POS-03 15:20 ZAL174 26 Feb 2016) 34 1532 Meantine Pole = 10.20 10.20 2 10.20 Food and Drink 10.20 10.20 VAT Net 2.50 8.50 20% VAT 2.70 20% VAT Total 10.20 10.20 10.20 10.20 1.70 Receipt 5.0 1 0.20 0207 494 0834 895 232'},
 {'content': ' Restaurant Name 3 Address 1 1 Address 2 3/15/12 6:06:44 PM 3 John 3 Table: 201 Sec. S Guest 1 1 #11 BLOODY MARY 5 40.75 FRESH OYSTER 1 12.50 553.25 5.33 664.60 664.60 0sI'},
 {'content': ' CITY of FALMOUTH 06/18/2003 PAYMENT BECEIPT 0123 0000 TAX 06/18/2003 MURBHY, CHRISTIAN E. Checck Number (s) : 001234 ACTC # M4751R / TRIH* 0019 02 02 629.95 02 Tx 0103 02 02 629.95 0103 05 05 29.53 1289.43 1289.43.00 1259.90.00 1259.90.00'}]

In [19]:
# generate ids for the data
for index, data in enumerate(file_data):
  ids.append(str(index+1))

In [20]:
ids

['1', '2', '3']

In [24]:
# creating the collection

multimodal_db.add(
    ids=ids,
    uris=uris,
    metadatas=metadatas

)

In [28]:
multimodal_db.count() # checking to confirm the elements have been inserting

3